In [1]:
import os
import ast
import time
import json
import pickle

import pandas as pd
import numpy as np
import tensorflow as tf

from math import floor, ceil
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

/opt/anaconda3/envs/PhetGPU/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.0.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
pwd

'/Users/cock/kDrive/PhD/Projects/Labs/blind-mitigation/notebooks/0_data_processing'

In [3]:
cd ../../data/flipped_classrooms/raw

/Users/cock/kDrive/PhD/Projects/Labs/blind-mitigation/data/flipped_classrooms/raw


# Roberta's data

In [4]:
with open('roberta_final_features.pkl', 'rb') as fp:
    roberta_features = pickle.load(fp)

In [5]:
with open('./vinitra_intermediate_features.pkl', 'rb') as fp:
    vinitra_features = pickle.load(fp)

In [6]:
roberta_features.shape

(215, 14, 82)

In [7]:
vinitra_features[0].shape

(214, 14, 85)

Performed a check, and the users are the same or at least seem to be in both dataframes.

# Id Matching

In [8]:
features = roberta_features[:-1]
labels = vinitra_features[1]
assert len(features) == 214
assert len(labels) == 214
# Last row is for the explainability network

In [9]:
# Matching of ids between vinitra's features's row and the id with the demographics
id_matching = pd.read_csv('./features/apr21-feature-mooc-flipped/user_id_mapping-epfl_algebrelineaire.csv')
id_matching.columns = ['user_row', 'user_id']
id_matching.head()

# demographics information from Vinitra, in the mooc folder in the server
demographics_all = pd.read_csv('./aied_flipped_paola_clusters.csv') 
demographics_all = demographics_all.rename(columns = {
    'e': 'user_id', 'o': 'country_diploma', 'k': 'gender'
})
display(demographics_all.head())

# Merging the row id, to the id, to the demographics data
print(len(id_matching))
demographics = id_matching.merge(demographics_all, how='left', on=['user_id'])
print(len(id_matching))

,student,profile,a,b,c,d,user_id,f,g,h,i,j,gender,l,m,n,country_diploma,p,q,r
0,0,B,4.50,0.0,0.0,13.0,ee38f02e4aefeea014b70358a50a3c42,46413,296767,Y2-2018-19,Flipped,4427T2597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y2-2018-19
1,1,D,4.50,0.0,0.0,13.0,3a50861aa4ceb79eff4def23a39c6ec5,48558,301739,Y2-2018-19,Flipped,4297T2518,M,IN,Diplôme HES,Suisse.Autres,Suisse,1.0,15.0,Y2-2018-19
2,2,B,5.25,0.0,0.0,12.0,883d63120d49df4f3cd1eac85be24696,48666,300667,Y2-2018-19,Flipped,3935T2301,M,SIE,Mat.Opt.Physique.Math,Suisse.PAM,Suisse,1.0,54.0,Y2-2018-19
3,3,B,4.50,0.0,0.0,11.0,a9e4c75be2065bad69e5556da9159555,46476,303808,Y2-2018-19,Flipped,4646T2727,F,CGC,Autre.Suisse,Suisse.Autres,Suisse,1.0,56.0,Y2-2018-19
4,4,E,4.75,0.0,1.0,8.0,3c7045987cd5d98c1c9b8960eedd5943,48870,298071,Y2-2018-19,Flipped,951T511,F,SV,Bacc. étranger,France,France,1.0,53.0,Y2-2018-19


214
214


In [10]:
# Check no demographics entries
demographics['nan_values'] = demographics[['gender', 'country_diploma']].isnull().sum(axis=1).tolist()
demographics[demographics['nan_values'] >= 1]



,user_row,user_id,student,profile,a,b,c,d,f,g,...,j,gender,l,m,n,country_diploma,p,q,r,nan_values
5,5,bb60fce06db2f82cb6a91f0583cc9b02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
14,14,991aeaa80650c5aa985698c5bfaae855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
15,15,955b205d7facf24d870a70c8c6e41c2f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
37,37,dbd27ac860fac8aea8b0de09d372a9aa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
47,47,37daf51c8a25c5b1bae7614c31c66715,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
56,56,ffb1bae8630b206879f2000d82e389d5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
58,58,b3a94d328838423f69a7117598b6f9e4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
60,60,14a6546489cfed0b9690efb5c3bed269,80.0,D,3.25,1.0,0.0,12.0,46485.0,302150.0,...,1354T752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y2-2018-19,2
77,77,c2d629e0fecc00783ff0f3279ad49d29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
83,83,dd9a851385f54250aad7c7006fe381e4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [11]:
demographics = demographics.fillna(-1)

# Formatting of the data

In [12]:
full_data = {}

j = 0
for i, row in (demographics.iterrows()):
    lid = row['user_id']
    lrow = row['user_row']
    target = labels[lrow]
    
    if row['gender'] == -1 or row['country_diploma'] == -1:
        continue
    
    
    student = {
        'sequence': features[lrow],
        'idx': lrow,
        'learner_id': lid,
        'gender': row['gender'],
        'country_diploma': row['country_diploma'],
        'pass': target,
        'cluster_aied_paola': row['profile']
    }
    full_data[j] = student
    j += 1
    
full_data['available_demographics'] = [
    'gender', 'country_diploma', 'cluster_aied_paola'
] 

In [13]:
with open('../features/roberta_vinitra_mirko_julien_features.pkl', 'wb') as fp:
    pickle.dump(full_data, fp)

In [14]:
len(full_data)

192